agenda :
__A*__

- cost / heuristic
- State class
    - state number
    - f, g, h
    - parent
- visited list
- to be visited queue
- 

-- Notes
- the heuristic function may not work properly as expected

In [1]:
from src.A_star.A_star import A_star

### Test

#### n=5

In [ ]:
ladders = [(9, 18), (14, 20)]
snakes = [(2, 12), (7, 16)]

n = 5
game = A_star(start_state=1, n=n, snakes=snakes, ladders=ladders, )
game.run()

In [ ]:
game.get_path()

[State1,
 State3,
 State5,
 State6,
 State8,
 State9,
 State18,
 State20,
 State21,
 State23,
 State25]

#### n=7

In [ ]:
ladders = [(9, 18), 
           (21, 35),
           (28, 42)]

snakes = [
    (5, 17),
    (19, 33),
    (25, 47),
]

n = 7

game = A_star(start_state=1, n=n, snakes=snakes, ladders=ladders, )
game.run()

In [ ]:
game.get_path()

[State1,
 State3,
 State5,
 State7,
 State8,
 State9,
 State18,
 State20,
 State21,
 State35,
 State36,
 State38,
 State40,
 State42,
 State43,
 State45,
 State46,
 State48,
 State49]

### final test 

In [ ]:
snakes = [(1,34), (5,25), (9,47) , (69,99),(57,87)]
ladders = [(3,51), (6,27), (20,70), (36,55),(63,95),(68,98)]

n = 10
game = A_star(start_state=1, n=n, snakes=snakes, ladders=ladders, )
game.run()

used a ladder
used a ladder


In [ ]:
game.get_path()

[State1,
 State3,
 State4,
 State56,
 State54,
 State88,
 State90,
 State91,
 State93,
 State95,
 State97,
 State99,
 State100]

In [ ]:
len(game.get_path())

13

In [ ]:
ladders = [(1, 24), (6, 99), (13, 55), (29, 71), (67, 88)]
snakes = [(8, 31), (15, 97), (66, 87), (8, 42)]

n = 10
game = A_star(start_state=1, n=n, snakes=snakes, ladders=ladders, )
game.run()

used a ladder
used a ladder


In [ ]:
game.get_path()

[State1,
 State24,
 State26,
 State28,
 State29,
 State71,
 State73,
 State75,
 State77,
 State79,
 State80,
 State81,
 State83,
 State85,
 State86,
 State88,
 State90,
 State91,
 State93,
 State95,
 State96,
 State98,
 State100]